In [ ]:
from pathlib import Path
from typing import List

import pydicom

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from kedro.extras.datasets.pickle import PickleDataSet
from kedro.config import ConfigLoader

import kornia.augmentation as K
import plotly.express as px

from PIL import Image

import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from torchvision.utils import save_image
from torchvision import transforms

from torch.autograd import Variable

In [ ]:
import os, sys
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../src/'))

from src.tagseg.data.acdc_dataset import AcdcDataSet
from src.tagseg.data.dmd_dataset import DmdDataSet, DmdTimeDataSet, Slice
from src.tagseg.pipelines.data_splitting.nodes import split_data

In [ ]:
conf_paths = ["../conf/base", "../conf/local"]
conf_loader = ConfigLoader(conf_paths)
conf_catalog = conf_loader.get("catalog*", "catalog*/**")

In [ ]:
params = {
    'filepath': '../data/03_primary/acdc_train.pt',
    'load_args': {'filepath_raw': '../data/01_raw/acdc-training', 'only_myo': True}
}

acdc = AcdcDataSet(**params)

In [ ]:
acdc.save(acdc.load())

In [ ]:
dataset = acdc.load()

In [ ]:
dataset = PickleDataSet(filepath='../' + conf_catalog['model_input']['filepath']).load()

In [ ]:
len(dataset)

In [ ]:
M, N = 20, 5
fig, ax = plt.subplots(M, N, figsize=(20, 100))

for i in range(M * N):
    m, n = i % M, i // M
    ax[m, n].imshow(dataset[i][0][0].cpu(), cmap='gray')
    ax[m, n].imshow(dataset[i][1].cpu(), cmap='Reds', alpha=0.3)
    ax[m, n].axis('off')